In [ ]:
# !pip install openai
# !pip install --upgrade openai
# !pip install python-dotenv

In [1]:
from llama_cpp import Llama
import os
import time
import openai
import pandas as pd
import json
from datasets import load_dataset 

In [2]:
!huggingface-cli login --token hf_
!huggingface-cli whoami

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\stefa\.cache\huggingface\token
Login successful
taoofstefan


In [3]:
from openai import OpenAI

client = OpenAI(
      api_key="Your-API-Key"
)

In [4]:
test = load_dataset('taoofstefan/function_call_weather', split='test')
print(test)

Dataset({
    features: ['Question', 'Answer'],
    num_rows: 100
})


In [5]:
df = pd.DataFrame(test)

In [6]:
# df = df.head(10)
df

,Question,Answer
0,What will be the peak temperature in Lagos today?,"{'type': 'function', 'function': {'name': 'get..."
1,How does the weather today in Sao Paulo compar...,"{'type': 'function', 'function': {'name': 'get..."
2,how warm is it in Kuala Lumpur right now?,"{'type': 'function', 'function': {'name': 'get..."
3,What is the UV index in Nairobi today?,"{'type': 'function', 'function': {'name': 'get..."
4,Is Lagos experiencing any extreme weather cond...,"{'type': 'function', 'function': {'name': 'get..."
...,...,...
95,Is there any rain expected in Rio de Janeiro w...,"{'type': 'function', 'function': {'name': 'get..."
96,What's the wind forecast for Hong Kong today?,"{'type': 'function', 'function': {'name': 'get..."
97,Will temperatures rise significantly in Gaboro...,"Weather conditions, including cold snaps and e..."
98,Are freezing temperatures expected in Budapest...,"{'type': 'function', 'function': {'name': 'get..."


In [7]:
tools = [{
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    }]

In [11]:
gpt_model = "gpt-4o-2024-05-13"
# gpt_model = "gpt-4-turbo-2024-04-09"
# gpt_model = "gpt-3.5-turbo-0125"

In [8]:
# Number of iterations
num_iterations = 1

In [9]:
system_message = "You are a helpful assistant with access to functions. Use the provided function to answer current weather questions."

In [12]:
# Loop through your dataset
for i in range(len(df)):
    # Your prompt or input from the dataset
    user_input = df.loc[i, 'Question']  # Adjust column name as needed
    
    for j in range(num_iterations):
        # Start timer for measuring inference time
        start = time.time()
        
        # Call OpenAI API for inference
        response = client.chat.completions.create(
            model=gpt_model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_input}
            ],
            functions=[tools[0]['function']], 
            function_call="auto"
        )
        
        # Extract the assistant's response based on finish_reason
        if response.choices[0].finish_reason == 'function_call':
            result = response.choices[0].message.function_call.arguments
        else:
            result = response.choices[0].message.content
        
        # Store the response in the DataFrame
        df.at[i, f'Result_{j}'] = result
        
        # End timer
        end = time.time()
        
        # Calculate duration and store it
        duration = end - start
        df.at[i, f'Inference_Time_{j}'] = duration

In [13]:
df['Result_0']

0               {"location":"Lagos","format":"celsius"}
1           {"location":"Sao Paulo","format":"celsius"}
2        {"location":"Kuala Lumpur","format":"celsius"}
3             {"location":"Nairobi","format":"celsius"}
4      {"location":"Lagos, Nigeria","format":"celsius"}
                            ...                        
95     {"location":"Rio de Janeiro","format":"celsius"}
96          {"location":"Hong Kong","format":"celsius"}
97    I currently don't have access to weather forec...
98    {"location":"Budapest, Hungary","format":"cels...
99              {"location":"Dubai","format":"celsius"}
Name: Result_0, Length: 100, dtype: object

In [14]:
# Dynamically set the output file name based on the model used
output_file_path = rf"Your-Path\result_openai_{gpt_model}.csv"

In [15]:
# Save to CSV with a specified delimiter
# Save the updated DataFrame with the results
df.to_csv(output_file_path, sep=";", index=False)

In [ ]:
df['Answer'][0]

In [ ]:
import ast

In [ ]:
answer_dict = ast.literal_eval(df['Answer'][0])

# Access the nested dictionary
parameters = answer_dict['function']['parameters']

print(parameters)

In [ ]:
import ast

def extract_parameters(answer):
    try:
        # Try to parse the string and extract the 'parameters' dictionary
        parsed_answer = ast.literal_eval(answer)
        return parsed_answer['function']['parameters']
    except (ValueError, SyntaxError, KeyError):
        # If there is an error, return the original answer
        return answer

# Create a new column 'Answer_Parameters' to store the 'parameters' dictionary or original answer
df['Answer_Parameters'] = df['Answer'].apply(extract_parameters)

# Display the DataFrame to verify the new column
print(df[['Answer', 'Answer_Parameters']])



In [ ]:
df.iloc[1]

In [ ]:
# Ensure both columns exist
if 'Result_0' in df.columns and 'Answer_Parameters' in df.columns:
    # Create a new column 'Comparison' to store the comparison results
    df['Comparison'] = df['Result_0'] == df['Answer_Parameters']

# Display the DataFrame to verify the new column
print(df[['Result_0', 'Answer_Parameters']])


In [ ]:
df.iloc[1]['Result_0']

In [ ]:
df.iloc[1]['Answer_Parameters']

In [ ]:
import ast
import json

# Function to convert JSON strings to dictionaries
def convert_to_dict(value):
    if isinstance(value, str):
        try:
            return json.loads(value)  # Try to parse as JSON
        except json.JSONDecodeError:
            try:
                return ast.literal_eval(value)  # Try to parse using ast.literal_eval
            except (ValueError, SyntaxError):
                return value  # Return the original value if parsing fails
    return value

# Convert both columns to dictionaries if needed
df['Result_0_dict'] = df['Result_0'].apply(convert_to_dict)
df['Answer_Parameters_dict'] = df['Answer_Parameters'].apply(convert_to_dict)

# Compare the converted columns
df['Comparison'] = df['Result_0_dict'] == df['Answer_Parameters_dict']

# Display the DataFrame to verify the new columns
print(df[['Result_0', 'Answer_Parameters', 'Result_0_dict', 'Answer_Parameters_dict', 'Comparison']])


In [ ]:
print(df['Comparison'] == True)

In [ ]:
# Filter the DataFrame to get the rows where Comparison is True
true_comparisons = df[df['Comparison'] == True]

# Print the filtered DataFrame
print(true_comparisons[['Comparison']])
